In [ ]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

In [2]:
import pyterrier as pt
pt.init()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')
duoT5 = DuoT5ReRanker(text_field='abstract')

In [4]:
dataset = pt.get_dataset("irds:cord19/trec-covid")
index_loc = "./index_path"
indexer = pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter(), fields=('title', 'abstract'))

[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
[INFO] [finished] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: [00:09] [269MB] [28.9MB/s]
[INFO] [starting] building docstore
docs_iter: 192509it [00:10, 18531.39it/s]
[INFO] [finished] docs_iter: [00:10] [192509it] [18529.42it/s]
[INFO] [finished] building docstore [10.39s]


10:22:34.417 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (8is9x9sc) - further warnings are suppressed

10:23:48.613 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 60 empty documents


In [5]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25") % 100
mono_pipeline = bm25 >> pt.text.get_text(dataset, "abstract") >> monoT5
duo_pipeline = mono_pipeline % 10 >> duoT5

In [6]:
pt.Experiment(
  [
   bm25,
   mono_pipeline,
   duo_pipeline,
  ],
  dataset.get_topics('title'),
  dataset.get_qrels(),
  names=["BM25", "BM25 >> monoT5", "BM25 >> monoT5 >> duoT5"],
  eval_metrics=["map", "recip_rank", "P.10", "ndcg_cut.10"]
)

[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/0307a37b6b9f1a5f233340a769d538ea
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [6.01MB/s]
[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/8138424a59daea0aba751c8a891e5f54
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [00:01] [1.14MB] [705kB/s]
[INFO] NumExpr defaulting to 2 threads.


Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


,name,map,recip_rank,P_10,ndcg_cut_10
0,BM25,0.076117,0.807094,0.678,0.601847
1,BM25 >> monoT5,0.081589,0.869000,0.740,0.676241
2,BM25 >> monoT5 >> duoT5,0.015089,0.817500,0.720,0.645854
